In [11]:
# !pip install torchsummary

In [12]:
##code
"""
Target : add more params in around 8k to check we can improve 
Result : test: 99.02%, train 99.28 at 15th rpoch 9,420
Analysis: training perfromance improved with more params from 98.61% to 99.02%
"""

'\nTarget : increase accuracy at 15th epoch, keeping same parameters\nResult : test: 99.37%, train 99.40 at 15th rpoch\nAnalysis: training speed improved, still very huge parameter\n'

In [13]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [14]:
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.convblock1 = nn.Sequential(
          nn.Conv2d(in_channels=1, out_channels=4 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(16),
          nn.ReLU(),
        )
        self.pool1 = nn.Sequential( ##>>0, 12
          nn.MaxPool2d(2,2),
        )
        self.convblock2 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=1, bias=False),
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(16),
          nn.ReLU(),        
        )
        self.convblock3 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(8),
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(16),
          nn.ReLU(),
        )
        self.pool2 = nn.Sequential( 
          nn.MaxPool2d(2,2),
        )
        self.convblock4 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=8 ,kernel_size=3,padding=1, bias=False), 
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=16 ,kernel_size=3,padding=1, bias=False), 
          nn.ReLU(),
          nn.Conv2d(in_channels=16, out_channels=10 ,kernel_size=3,padding=0, bias=False), 
          # nn.ReLU()
        )

    def forward(self, x):
        x = self.convblock1(x)
        # print (x.shape)
        x = self.pool1(x)
        # print (x.shape)
        x= self.convblock2(x)
        # print (x.shape)
        x= self.convblock3(x)
        # print (x.shape)
        x= self.pool2(x)
        # print (x.shape)
        x= self.convblock4(x)
        # print (x.shape)
        x = x.view(-1,10)
        x = F.log_softmax(x)
        return x 


In [15]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              36
       BatchNorm2d-2            [-1, 4, 26, 26]               8
              ReLU-3            [-1, 4, 26, 26]               0
            Conv2d-4            [-1, 8, 24, 24]             288
       BatchNorm2d-5            [-1, 8, 24, 24]              16
              ReLU-6            [-1, 8, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           1,152
       BatchNorm2d-8           [-1, 16, 22, 22]              32
              ReLU-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11            [-1, 4, 11, 11]             576
      BatchNorm2d-12            [-1, 4, 11, 11]               8
             ReLU-13            [-1, 4, 11, 11]               0
           Conv2d-14            [-1, 8,

/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_52471/2548857420.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [16]:
transforms_ = transforms.Compose([
                        transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

In [17]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 7, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [18]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1)
        correct+=pred.eq(target).sum().item()
    acc = 100*correct/len(train_loader.dataset)
    print(f'Train Accuracy: {acc:.2f}%')
        
def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _,(data, target) in enumerate(test_loader):
            data, target = data.float().to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.5)
for epoch in range(1, 15):
    print(f'Epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)


Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_52471/2548857420.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
loss=0.19353020191192627 batch_id=937: 100%|██████████| 938/938 [00:42<00:00, 21.90it/s] 


Train Accuracy: 86.73%
Test set: Average loss: 0.1011, Accuracy: 9692/10000 (96.92%)
Epoch: 2


loss=0.1569654494524002 batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.52it/s]   


Train Accuracy: 97.57%
Test set: Average loss: 0.0734, Accuracy: 9776/10000 (97.76%)
Epoch: 3


...

Train Accuracy: 99.25%
Test set: Average loss: 0.0409, Accuracy: 9881/10000 (98.81%)
Epoch: 13


loss=0.04892423748970032 batch_id=937: 100%|██████████| 938/938 [00:26<00:00, 35.09it/s]   


Train Accuracy: 99.28%
Test set: Average loss: 0.0342, Accuracy: 9902/10000 (99.02%)
Epoch: 14


loss=0.22233207523822784 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 31.61it/s]   


Train Accuracy: 99.35%
Test set: Average loss: 0.0340, Accuracy: 9896/10000 (98.96%)
